In [1]:
file_path = r'C:\UCD\RESEARCH\Database DB\rt_leavetimes_2016_I_DB.txt'

In [2]:
rt_lt_16 = spark.read.option("delimiter",";").csv(file_path,header=True)

In [3]:
from_pattern="dd-MMM-yy HH:mm:ss"
to_pattern="EEEE"
from pyspark.sql.functions import unix_timestamp,from_unixtime

In [4]:
rt_lt_16_dow = rt_lt_16.withColumn('dayOfWeek', from_unixtime(unix_timestamp(rt_lt_16['dayofservice'], from_pattern), to_pattern))

In [5]:
rt_lt_16_Saturday = rt_lt_16_dow.where(rt_lt_16_dow.dayOfWeek=='Saturday')

In [6]:
file_path = r'C:\UCD\RESEARCH\Database DB\rt_leavetimes_2017_I_DB.txt'

In [7]:
rt_lt_17 = spark.read.option("delimiter",";").csv(file_path,header=True)

In [8]:
rt_lt_17_dow = rt_lt_17.withColumn('dayOfWeek', from_unixtime(unix_timestamp(rt_lt_17['dayofservice'], from_pattern), to_pattern))

In [9]:
from functools import reduce
from pyspark.sql import DataFrame
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [10]:
df_Full_LT = unionAll(rt_lt_16_dow,rt_lt_17_dow)

In [11]:
# df_Full_LT.write.partitionBy("dayOfWeek").csv(r'C:\UCD\RESEARCH\Database DB\filesSplit')

In [12]:
# rt_lt_16_Saturday.count()

In [13]:
# df_Full_LT.groupBy("dayOfWeek").count().show()

In [14]:
# rt_lt_16_Saturday.coalesce(1).write.csv(r'C:\UCD\RESEARCH\Database DB\Saturday.csv')

In [15]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [16]:
# define the sample size
percent_back = 0.05

In [17]:
#This calculates the fraction of population that has to be included in the sample
# 5% from each weekday 
frac = dict(
    (e.dayOfWeek, percent_back) 
    for e 
    in df_Full_LT.select('dayOfWeek').distinct().collect()
)

In [18]:
# sample the data
sampled = df_Full_LT.sampleBy('dayOfWeek', fractions=frac)

In [19]:
sampled_total_count = sampled.count()

In [20]:
sampled_counts = sampled.groupBy('dayOfWeek').count()

In [21]:
# original_total_count = df_Full_LT.count()

In [22]:
print(sampled_total_count)

4605864


In [23]:
type(sampled_counts)

pyspark.sql.dataframe.DataFrame

In [24]:
sampled_counts.show()

+---------+------+
|dayOfWeek| count|
+---------+------+
|Wednesday|782243|
|  Tuesday|801147|
|   Friday|723407|
| Thursday|703069|
| Saturday|519245|
|   Monday|719555|
|   Sunday|357198|
+---------+------+



In [25]:
# print(original_total_count)

In [26]:
# df_Full_LT.groupBy('dayOfWeek').count().show()

In [27]:
# sampled.repartition(1).write.partitionBy("dayOfWeek").csv(r'C:\UCD\RESEARCH\Database DB\Splits_Sample_Partitioned')

In [28]:
sampled.columns

['datasource',
 'dayofservice',
 'tripid',
 'progrnumber',
 'stoppointid',
 'plannedtime_arr',
 'plannedtime_dep',
 'actualtime_arr',
 'actualtime_dep',
 'vehicleid',
 'passengers',
 'passengersin',
 'passengersout',
 'distance',
 'suppressed',
 'justificationid',
 'lastupdate',
 'note',
 'dayOfWeek']

In [29]:
frac

{'Wednesday': 0.05,
 'Tuesday': 0.05,
 'Friday': 0.05,
 'Thursday': 0.05,
 'Saturday': 0.05,
 'Monday': 0.05,
 'Sunday': 0.05}